STEP 1

In [17]:
import pandas as pd

data = pd.read_excel("data.xlsx")


In [18]:
data

,SrNo,Patient_Id,Symptoms - 1,Unnamed: 3,Unnamed: 4,DOB,Symptoms - 2,Unnamed: 7,Unnamed: 8,Symptoms - 3,Unnamed: 10,Unnamed: 11,Other Symptoms
0,NaN,NaN,Fever,NaN,NaN,NaN,Cough,NaN,NaN,Cold,NaN,NaN,Text
1,NaN,NaN,Mild,Low,High,NaN,Mild,Low,High,Mild,Low,High,Text
2,1.0,1.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Body Ache
3,2.0,2.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Shivering
4,3.0,3.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Head Ache
5,4.0,4.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Nausia
6,5.0,5.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Sickness
7,6.0,6.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Vertigo
8,7.0,7.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Head Ache
9,8.0,8.0,Y,N,N,NaN,N,Y,Y0,N,N,Y,Nausia


In [19]:
symptoms = {
    'Symptom1': {'Fever': ['Mild', 'Low', 'High']},
    'Symptom2': {'Cough': ['Mild', 'Low', 'High']},
    'Symptom3': {'Cold': ['Mild', 'Low', 'High']}
}

print(symptoms)

{'Symptom1': {'Fever': ['Mild', 'Low', 'High']}, 'Symptom2': {'Cough': ['Mild', 'Low', 'High']}, 'Symptom3': {'Cold': ['Mild', 'Low', 'High']}}


STEP 2

In [20]:
# Extract unique symptoms from the "Other_Symptoms" column
other_symptoms = df['Other Symptoms'].str.split(',').explode().str.strip().unique()

# Prepare a list to track any new symptoms (those not already found in the dictionary)
loss = []

# Check each symptom in "Other_Symptoms" against the existing symptoms in the dictionary
for symptom in other_symptoms:
    found = False
    # Iterate over each symptom group in the existing dictionary
    for symptom_group, symptom_info in symptoms.items():
        if symptom in symptom_info:
            found = True
            break
    # If the symptom is not found in the existing dictionary
    if not found:
        # Add the symptom to the "loss" list
        loss.append(symptom)
        # Create a new symptom group for the newly found symptom
        new_symptom_group = f'Symptom{len(symptoms) + 1}'
        symptoms[new_symptom_group] = {symptom: None}

# Print the updated symptoms dictionary
print("Updated Symptoms Dictionary:")
for symptom_group, symptom_info in symptoms.items():
    print(f"{symptom_group}: {symptom_info}")

# Print the list of new symptoms that were added
print("\nLoss (new symptoms added to the dictionary):")
print(loss)


Updated Symptoms Dictionary:
Symptom1: {'Fever': ['Mild', 'Low', 'High']}
Symptom2: {'Cough': ['Mild', 'Low', 'High']}
Symptom3: {'Cold': ['Mild', 'Low', 'High']}
Symptom4: {'Text': None}
Symptom5: {'Body Ache': None}
Symptom6: {'Shivering': None}
Symptom7: {'Head Ache': None}
Symptom8: {'Nausia': None}
Symptom9: {'Sickness': None}
Symptom10: {'Vertigo': None}

Loss (new symptoms added to the dictionary):
['Text', 'Body Ache', 'Shivering', 'Head Ache', 'Nausia', 'Sickness', 'Vertigo']


In [21]:
# Calculate the total number of symptom groups (columns) in the dictionary
total_columns = len(symptoms)

# Calculate the ratio of new symptoms (loss) compared to total symptom groups
loss_number = len(loss) / total_columns if total_columns > 0 else 0

# Print the calculated loss as a number
print("\nLoss as a number:", loss_number)



Loss as a number: 0.7


STEP 3

In [22]:
# Iterate through each symptom group and its symptoms
for symptom_group, symptom_info in symptoms.items():
    for symptom, severity in symptom_info.items():
        # If the severity is missing or is just a string, update it
        if severity is None or isinstance(severity, str):
            symptoms[symptom_group][symptom] = ['Mild', 'Low', 'High']

# Print the updated symptom dictionary with severity levels
print("Updated Symptoms Dictionary with Severity Levels:")
for symptom_group, symptom_info in symptoms.items():
    print(f"{symptom_group}: {symptom_info}")


Updated Symptoms Dictionary with Severity Levels:
Symptom1: {'Fever': ['Mild', 'Low', 'High']}
Symptom2: {'Cough': ['Mild', 'Low', 'High']}
Symptom3: {'Cold': ['Mild', 'Low', 'High']}
Symptom4: {'Text': ['Mild', 'Low', 'High']}
Symptom5: {'Body Ache': ['Mild', 'Low', 'High']}
Symptom6: {'Shivering': ['Mild', 'Low', 'High']}
Symptom7: {'Head Ache': ['Mild', 'Low', 'High']}
Symptom8: {'Nausia': ['Mild', 'Low', 'High']}
Symptom9: {'Sickness': ['Mild', 'Low', 'High']}
Symptom10: {'Vertigo': ['Mild', 'Low', 'High']}


In [23]:
# Determine the total number of symptom groups (columns)
total_columns = len(symptoms)

# Calculate the ratio of new symptoms (loss) compared to the total number of symptom groups
loss_number = len(loss) / total_columns if total_columns > 0 else 0

# Print the loss as a numerical value
print("\nLoss as a number:", loss_number)



Loss as a number: 0.7


In [24]:
import json
with open("symptoms_dict.json", 'w') as f:
        json.dump(symptoms, f, indent=4)

STEP 4

In [31]:
import pandas as pd
import json
import xml.etree.ElementTree as ET

class SymptomsParser:
    def __init__(self, dict_file='symptoms_dict.json'):
        self.symptoms_dict = {}
        self.dict_file = dict_file
        self.load_dictionary()

    # Read data from different file formats
    def read_data(self, file_path):
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path)
        elif file_path.endswith('.tsv'):
            return pd.read_csv(file_path, delimiter='\t')
        elif file_path.endswith('.json'):
            return pd.read_json(file_path)
        elif file_path.endswith('.xml'):
            return self.parse_xml(file_path)
        elif file_path.endswith('.xlsx'):
            return pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format")

    # Parse XML file into DataFrame
    def parse_xml(self, file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        data = []
        for patient in root.findall('./Patient'):
            row = {child.tag: child.text for child in patient}
            data.append(row)
        return pd.DataFrame(data)

    # Add new symptoms from data to the dictionary
    def enhance_dictionary(self, data, symptom_col='Other Symptoms'):
        # Ensure column names are trimmed of extra spaces and normalized
        data.columns = data.columns.str.strip()

        if symptom_col not in data.columns:
            raise KeyError(f"The column '{symptom_col}' does not exist in the data.")

        for symptoms in data[symptom_col].dropna():
            for symptom in symptoms.split(','):
                symptom = symptom.strip()
                if not any(symptom in group for group in self.symptoms_dict.values()):
                    self.symptoms_dict[f'Symptom{len(self.symptoms_dict) + 1}'] = {symptom: ['Mild', 'Low', 'High']}
        return self.symptoms_dict

    # Print filtered data based on symptoms in the dictionary
    def print_filtered_data(self, data, symptom_col='Other Symptoms'):
        available_symptoms = [symptom for group in self.symptoms_dict.values() for symptom in group]
        filtered_data = data[data[symptom_col].apply(lambda x: any(symptom in x for symptom in available_symptoms) if pd.notnull(x) else False)]
        print("Filtered Data Based on Symptoms in Dictionary:")
        print(filtered_data)

    # Save the dictionary to a file
    def dump_dictionary(self):
        with open(self.dict_file, 'w') as f:
            json.dump(self.symptoms_dict, f, indent=4)
        print(f"Dictionary saved to {self.dict_file}")

    # Load the dictionary from a file if it exists
    def load_dictionary(self):
        try:
            with open(self.dict_file, 'r') as f:
                self.symptoms_dict = json.load(f)
        except FileNotFoundError:
            print(f"No dictionary found at {self.dict_file}. Starting with an empty dictionary.")
            self.symptoms_dict = {}

    # Manually update the dictionary
    def manual_update(self):
        symptom = input("Enter a new symptom: ")
        severity = input("Enter severity levels (comma-separated, e.g., Mild, Low, High): ").split(',')
        severity = [level.strip() for level in severity]

        if symptom not in [s for group in self.symptoms_dict.values() for s in group]:
            self.symptoms_dict[f'Symptom{len(self.symptoms_dict) + 1}'] = {symptom: severity}
            print(f"Added '{symptom}' with levels {severity} to the dictionary.")
            self.dump_dictionary()
        else:
            print(f"Symptom '{symptom}' already exists in the dictionary.")

    # Process the data and update the dictionary
    def process_data(self, file_path, symptom_col='Other Symptoms'):
        # Step 1: Read data
        data = self.read_data(file_path)

        # Step 2: Normalize column names and ensure the required column exists
        data.columns = data.columns.str.strip()
        if symptom_col not in data.columns:
            raise KeyError(f"Column '{symptom_col}' is not present in the file.")

        # Step 3: Print filtered data based on available symptoms in the dictionary
        self.print_filtered_data(data, symptom_col)

        # Step 4: Enhance dictionary with new symptoms from the dataset
        self.enhance_dictionary(data, symptom_col)

        # Step 5: Reprint filtered data based on the updated dictionary
        self.print_filtered_data(data, symptom_col)

        # Step 6: Save the updated dictionary
        self.dump_dictionary()

# Usage example
if __name__ == "__main__":
    parser = SymptomsParser(dict_file='symptoms_dict.json')
    file_path = 'data.xlsx'  # Change this to your actual file path

    try:
        parser.process_data(file_path, symptom_col='Other Symptoms')

        # Ask if the user wants to manually update the dictionary
        if input("Do you want to manually update the dictionary? [Y/N]: ").upper() == "Y":
            parser.manual_update()
    except KeyError as e:
        print(f"Error: {e}")


Filtered Data Based on Symptoms in Dictionary:
    SrNo  Patient_Id Symptoms - 1 Unnamed: 3 Unnamed: 4  DOB Symptoms - 2  \
0    NaN         NaN        Fever        NaN        NaN  NaN        Cough   
1    NaN         NaN         Mild        Low       High  NaN         Mild   
2    1.0         1.0            Y          N          N  NaN            N   
3    2.0         2.0            Y          N          N  NaN            N   
4    3.0         3.0            Y          N          N  NaN            N   
5    4.0         4.0            Y          N          N  NaN            N   
6    5.0         5.0            Y          N          N  NaN            N   
7    6.0         6.0            Y          N          N  NaN            N   
8    7.0         7.0            Y          N          N  NaN            N   
9    8.0         8.0            Y          N          N  NaN            N   
10   9.0         9.0            Y          N          N  NaN            N   
11  10.0        10.0         